# MQTT

[MQTT](https://mqtt.org/) is an efficient protocol for communication between IoT devices. 

## Broker

*ide49* comes with a built-in [mosquitto](https://mosquitto.org/) MQTT broker. By default it is configured to accept connections on ports 1883 and 9001 (websockets). Modify the configuration at `/service-config/mosquitto/config/` to enable additional features such as encryption and passwords.

## Clients

Below we show several examples of MQTT clients. The *Web Client* and *Command Line* tools are mostly useful for debugging. The *CPython* and *MicroPython* are examples of what you might incorporate in an application.

### Web Client

Web clients are great to observe diagnose MQTT communication. Below is an example using the [HiveMQ client](http://www.hivemq.com/demos/websocket-client/). Change the `Host` field below to the URL of your instance of *ide49* (e.g. `https://iot49.local/`) and make sure `Port` is set to 9001. The `ClientID` is a random string generated automatically and must be different for evey client connecting to the broker. 

```{figure} figures/hivemq_connect.png
:width: 500px
```

Click connect and add subscriptions and publish messages. The example below publishes and subscribes to the same topic. Normally these would differ to communicate with other MQTT clients.

```{figure} figures/hivemq_publish.png
:width: 500px
```

### Command Line

Alternatively use [mosquitto_sub](https://mosquitto.org/man/mosquitto_sub-1.html) and [mosquitto_pub](https://mosquitto.org/man/mosquitto_pub-1.html) in a terminal window or from a notebook like in the examples below:

In [3]:
!mosquitto_sub -h 10.39.40.200 -t "testtopic/#"

Hello, how are you today?
Great to hear! I'm happy for you.
^C


In [4]:
!mosquitto_pub -h 10.39.40.200 -t "testtopic/hello" -m "Grüezi"

### CPython

A common use case of MQTT is communication between microcontrollers and host computers. Below is a simple example receiving and sending MQTT messages. It first publishes a few messages to `testtopic` and then waits for 20 seconds for messages published to `testpub`. Use the web client to command line discussed above to listen and send to these topics.

Before running the code below, update the broker address.

In [7]:
%%host

from paho.mqtt.client import Client
from threading import Event
import time

USE_WS = False
BROKER = '10.39.40.200'

if USE_WS:
    PORT = 9001
    TRANSPORT = 'websockets'
else:
    PORT = 1883
    TRANSPORT = 'tcp'

# on_message runs on a different thread ...
connected = Event()

def on_message(client, userdata, message):
    print(f"message received: {message.topic} = {message.payload}")

def on_connect(client, userdata, flags, rc):
    global connected
    client.subscribe('testpub/#')
    connected.set()
    
def on_disconnect(client, userdata, rc):
    global connected
    connected.clear()
    print(f"disconnected, rc={rc}") 
    
client = Client("client-socks", transport=TRANSPORT)
client.on_message = on_message
client.on_connect = on_connect
client.on_disconnect = on_disconnect
client.connect(BROKER, PORT)

# start a separate thread listening to messages
client.loop_start()

# wait for connection before publishing
connected.wait()

for i in range(5):
    topic = f"testtopic/topic_{i}"
    msg   = f"msg {i}"
    print(f"publish {topic} = {msg}")
    client.publish(topic, msg)
    time.sleep(0.1)

# keep listening to messages
time.sleep(20)

client.disconnect()
print()

publish testtopic/topic_0 = msg 0
publish testtopic/topic_1 = msg 1
publish testtopic/topic_2 = msg 2
publish testtopic/topic_3 = msg 3
publish testtopic/topic_4 = msg 4
message received: testpub/1 = b"Great to hear! I'm happy for you."
message received: testpub/1 = b'As always'

disconnected, rc=0


### MicroPython

Several MQTT clients are available for MicroPython. The sample below uses a simple client from the "official" library, [micropython-lib](https://github.com/micropython/micropython-lib).

Let's first download the code:

In [7]:
%%bash

src=https://raw.githubusercontent.com/micropython/micropython-lib/master/micropython/umqtt.simple/umqtt/simple.py
dest_dir=$IOT_PROJECTS/internet/code/lib/

mkdir -p $dest_dir
curl -o $dest_dir/mqtt_client.py $src

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6479  100  6479    0     0  17510      0 --:--:-- --:--:-- --:--:-- 17463


In [7]:
%connect esp32 -q
%rsync

UPDATE  /lib/mqtt_client.py


The examples below are taken from the [github archive](https://github.com/micropython/micropython-lib/tree/master/micropython/umqtt.simple). Set up a client (e.g. HiveMQ) to check them out.

In [1]:
from mqtt_client import MQTTClient

# Test e.g. with:
# mosquitto_sub -h 10.39.40.200 -t testtpic

client = MQTTClient('umqtt_client', '10.39.40.200')
client.connect()
client.publish(b"testtopic", b"Hello from micropython!")
client.disconnect()

This example subscribes to topic `led` (note that topics and messages are `bytes`, not strings) to control an LED. Change the LED pin number to match your board.

In [1]:
from mqtt_client import MQTTClient
from machine import Pin
import ubinascii
import machine
import micropython

led = Pin(13, Pin.OUT, value=1)

def sub_cb(topic, msg):
    global quit
    print("topic {} = {}".format(topic.decode(), msg.decode()))
    if msg == b"on":
        led.value(0)
    elif msg == b"off":
        led.value(1)
    elif msg == b"toggle":
        led.value(1-led.value())
    elif msg == b"quit":
        quit = True

client = MQTTClient('mqtt_led_client', '10.39.40.200')

# Subscribed messages will be delivered to this callback
client.set_callback(sub_cb)

client.connect()
client.subscribe(b'led')

quit = False

try:
    while not quit:
        # call this to receive messages
        client.wait_msg()
except Exception as e:
    print("*****", e)
finally:
    client.disconnect()

topic led = toggle
topic led = toggle
topic led = toggle
topic led = toggle
topic led = off
topic led = quit


Note that it is necessary to call `client.wait_msg()` to receive messages, i.e. place this instruction somewhere in your program where it will be executed regularly.

In simple programs like the example above that's easy enough to do, but in more complex applications this may not be trivial. 

An alternative is to use an mqtt client that uses the `asyncio`, a cooperative multitasking framework. An asyncronous MQTT client for MicroPython is available [here](https://github.com/peterhinch/micropython-mqtt/tree/master/mqtt_as).